# E.T.L  (Extract Transform Load)
En esta oportunidad se realizara la `Extraccion, Transformacion y Carga` (E.T.L por sus siglas en ingles), con el objetivo de encontrar patrones y tendencias que sean de ayuda al momento de realizar conclusiones

## Importar librerias a utilizar

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import glob

## Metadata Sitios

En esta ocasion contamos con gran cantidad de datos en diversos archivos `.json`, por ende, para facilitar la deteccion de nulos e irregularidades, se uniran todos los datos en un solo *dataframe*

- Combinamos todos los archivos `.json` para detectar alteraciones en los datos y luego exportarlos como un unico archivo

In [ ]:
# Ruta donde están almacenados los archivos JSON
path = "../../Datasets/Sitios"

# Usamos glob para obtener todos los archivos JSON en el directorio
all_files = glob.glob(os.path.join(path, "*.json"))

df=[]
for file in all_files:
    archivo = pd.read_json(file,lines=True,encoding="utf-8")
    df.append(archivo)
    
df_combined = pd.concat(df, ignore_index=True)

In [ ]:
# Visualizacion de algunos datos del dataset combinado
print(df_combined.shape)
df_combined.head(2)

In [23]:
df_combined.isna().sum()

name                     37
address               80511
gmap_id                   0
description         2770722
latitude                  0
longitude                 0
category              17419
avg_rating                0
num_of_reviews            0
price               2749808
hours                787405
MISC                 690834
state                746455
relative_results     295058
url                       0
dtype: int64

### Observaciones
- Bajo el anterior analisis determinamos que los campos `description`, `price` y `state` cuentan con una gran cantidad de nulos.
- Por otro lado, columnas como `hours` no seran tenidas en cuenta.

- Eliminacion de columnas no utilizadas

In [6]:
df_combined.drop(columns=["description","hours","state","price","MISC"],inplace=True)

In [7]:
df_combined.head(2)

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.35710,[Pharmacy],4.9,16,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.21529,[Textile exporter],4.5,6,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


- Chequeamos que no haya registros <i>duplicados</i>

In [19]:
df_combined = df_combined[~df_combined["gmap_id"].duplicated()]

- Exportamos los datos

In [8]:
df_combined.to_parquet("../../Datasets/final-ds/sitios_combined_full",engine="pyarrow")